In [1]:
!pip install earthengine-api geopandas rasterio folium


  Using cached geopandas-1.0.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached rasterio-1.4.3-cp312-cp312-win_amd64.whl.metadata (9.4 kB)
  Using cached folium-0.19.4-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached pyogrio-0.10.0-cp312-cp312-win_amd64.whl.metadata (5.6 kB)
  Using cached pyproj-3.7.1-cp312-cp312-win_amd64.whl.metadata (31 kB)
  Using cached shapely-2.0.7-cp312-cp312-win_amd64.whl.metadata (7.1 kB)
  Using cached affine-2.4.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached cligj-0.7.2-py3-none-any.whl.metadata (5.0 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl.metadata (6.4 kB)
  Using cached branca-0.8.1-py3-none-any.whl.metadata (1.5 kB)
  Using cached xyzservices-2025.1.0-py3-none-any.whl.metadata (4.3 kB)
Using cached geopandas-1.0.1-py3-none-any.whl (323 kB)
Using cached rasterio-1.4.3-cp312-cp312-win_amd64.whl (25.4 MB)
Using cached folium-0.19.4-py2.py3-none-any.whl (110 kB)
Using cached branca-0.8.1-py3-none-any.whl (26 kB)
Using cached c


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Import packages
import ee
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt

# Authenticate and initialize GEE
ee.Authenticate()
ee.Initialize()

# Define ROI for Punjab, India
roi = ee.Geometry.Rectangle([74.5, 29.5, 76.5, 31.5])

# Load Sentinel-2 collection
collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterBounds(roi) \
    .filterDate('2023-01-01', '2023-03-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# Apply cloud masking
def mask_clouds(image):
    qa = image.select('QA60')
    cloud_mask = qa.bitwiseAnd(1 << 10).eq(0)
    return image.updateMask(cloud_mask)

collection = collection.map(mask_clouds)

# Calculate NDVI
def calculate_ndvi(image):
    nir = image.select('B8')  # Near-Infrared band
    red = image.select('B4')  # Red band
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')  # NDVI formula
    return image.addBands(ndvi)

ndvi_collection = collection.map(calculate_ndvi)

# Export NDVI image (median composite)
task = ee.batch.Export.image.toDrive(
    image=ndvi_collection.median(),  # Median composite of the collection
    description='NDVI_Punjab_2023',
    scale=10,  # 10m resolution
    region=roi,
    fileFormat='GeoTIFF',
    maxPixels=1e13
)

# Start the export task
task.start()

# Check task status
print(task.status())

{'state': 'READY', 'description': 'NDVI_Punjab_2023', 'priority': 100, 'creation_timestamp_ms': 1740221543327, 'update_timestamp_ms': 1740221543327, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'JDX5S4PV5M35I36SM7FLHRTK', 'name': 'projects/730570924758/operations/JDX5S4PV5M35I36SM7FLHRTK'}
